In [6]:
import scipy.io as sio
d1T = sio.loadmat('BBCIData/S14T.mat')
d1E = sio.loadmat('BBCIData/S14E.mat')

In [9]:
d1T['data'].shape

(1, 5)

In [77]:
len(d1T['data'][0]) # this returns 5 items
d1T['data'][0] # 5 runs of 20 trials
d1E['data'][0] # 3 runs of 20 trials


array([ array([[ (array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), array([[  5121,  10435,  15725,  21315,  26866,  32408,  37675,  43213,
         48759,  53995,  59583,  65138,  70529,  76128,  81422,  86817,
         92222,  97814, 103036, 108423]], dtype=int32), array([[2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2]], dtype=uint8), array([[512]], dtype=uint16), array([[array(['right hand'],
      dtype='<U10'),
        array(['feet'],
      dtype='<U4')]], dtype=object))]],
      dtype=[('X', 'O'), ('trial', 'O'), ('y', 'O'), ('fs', 'O'), ('classes', 'O')]),
       array([[ (array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...

In [83]:
d1E['data'][0][0][0][0][0].shape    # 220 seconds (whole run) and 15 channels

(112640, 15)

In [84]:
d1E['data'][0][0][0][0][1] # timestamps

array([[  5121,  10495,  15692,  21258,  26655,  31942,  37206,  42611,
         47800,  53323,  58865,  64231,  69501,  74907,  80129,  85574,
         91048,  96399, 101734, 106918]], dtype=int32)

In [85]:
d1E['data'][0][0][0][0][2] # labels

array([[2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2]], dtype=uint8)

In [86]:
d1E['data'][0][0][0][0][3] # sampling rate

array([[512]], dtype=uint16)

In [87]:
d1E['data'][0][0][0][0][4] # meta-data

array([[array(['right hand'],
      dtype='<U10'),
        array(['feet'],
      dtype='<U4')]], dtype=object)